# Оптимизация параметров устройства при изготовлении шар-баллона при помощи нейронной сети

ВКР Кудрявцева А.В. - МИСиС ИТКН МИВТ-2020-8-16

## Состав и последовательность работы:

1.Постановка и описание задачи

2.Загрузка (установка) библиотек и модулей

3.Инициализация входных данных

4.Подготовка данных для работы с учётом зависимостей

5.Построение модели

6.Обучение модели для поиска зависимостей (с учителем, регрессионнный анализ)

7.Использование обученной модели для максимизации выходных данных от входных

8.Визуализация

9.Анализ результата и выводы

## 1.Постановка и описание задачи

Методом электронно-лучевой наплавки проволоки из сплава титана ВТ6 в вакууме на затравке (полдожке) послойно наплавляется изделие в виде полусферы. Далее после окончания наплавки изделие остывает, механически обрабатывается и сваривается с аналогичным изделием, в результате чего получается шар-баллон, который далее используется в разных устройствах и изделиях, для работы под давлением. Перед началом наплавки создаётся 3D модель, в которой задаются основные геометрические параметры, а также получается приближённая оценка механических характеристик будущего изделия таких как прочность, упругость и другие. Однако, такая оценка механических характеристик не учитывает некоторые параметры изготовления, приводящие в отклонениям при изготовлении полусферы шар-баллона, таким как тепловые деформации начальных слоёв, пористость поверхности, параметры прочности, ниже заявленных. Задача состоит в том, чтобы найти зависимости температур тела изделия в разных точках в процессе изготовления от таких входных параметров как ускоряющее напряжение, сила тока фокусировки, скорость наплавки, сорость подачи проволоки, толщина стенки изделия, с тем, чтобы в дальнейшем подбирать такие режимы изменения входных параметров. Это возможно позволит находить оптимальные температурные режимы наплавки. В конечном итоге, необходимо подтвердить или опровергнуть гипотезу, что возможно смоделировать процессы термического упрочнения изделия прямо в процессе производства за счёт контроля и задания температур во время наплавки с помощью входных параметров.

## 2.Загрузка и установка библиотек

In [1]:
!pip install bokeh -qq

In [2]:
# base
import random as rd
import pandas as pd
import numpy as np
import random 
import os
import warnings

#sklearn
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split

import math
from itertools import product

#scipy
from scipy.optimize import *
from scipy.integrate import quad
import matplotlib.pyplot as plt 
from scipy.interpolate import griddata

# visualize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter

#torch
import torch

# TF
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model

# подключаем garbage collector
import gc


from sklearn.linear_model import SGDRegressor
SEED = 42
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

import numpy as np; np.random.seed(SEED)
import seaborn as sns; sns.set_theme()

from scipy.stats import skewnorm  

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#parallel
from joblib import Parallel, delayed
import multiprocessing

# интересная статья с инстурментами для визуализации 
# https://mode.com/blog/python-data-visualization-libraries/
# bokeh for visualization
import bokeh
from bokeh.io import show
from bokeh.palettes import Turbo256
from bokeh.plotting import figure
from bokeh.transform import linear_cmap

# uuid generation
import uuid

#MLP
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html
from sklearn.neural_network import MLPRegressor

Instructions for updating:
non-resource variables are not supported in the long term


## 3. Анализ данных (EDA)

## 3.1 Инициализация входных данных

Заказчиком были переданы следующие данные:

UsNapr=7000 !ускоряющее напряжение, В

SilaToka=90e-3 !сила тока, А

PPuchka=UsNaprSilaToka0.95*16 !мощность пучка при развертке цилиндрической

DLucha=15e-3 !диаметр развертки луча, м

elemPerLuch=2 !число элементов в развертке луча, должно быть больше 1

V=8e-3 !м/сек скорость наплавки

delta=0.8e-3 !высота элемента в м - constanta

ElStep=DLucha/elemPerLuch = 75e-4 !размер геометрического елемента по - окружности, в м

R_1=250e-3 !внутренний диаметр сферы - constanta

T_1=15e-3 !толщина стенки сферы - constanta

predel_ug=90 !Угол, до которого сфера разбивается на слои

DZatravki=135e-3 !Диаметр затравки, м

HZatravki=60e-3 !Высота затравки, м

HKoroni=90e-3 !Высота короны (должна быть обязательно)

HNachalaRosta=10e-3 !Высота начала роста полусферы от торца затравки

TimeShift = 6 !Время наплавки во время смены, час

TimeSverhUroch= 2 !Время наплавки сверхурочно, в случае неуспевания, час - constanta

TimeOstiv=10 !Время остывания между сменами,час - constanta

TimePeredIzvlech=3 !Время остывания перед извлечением, час - constanta

TimeProgrev=0.3 !Время прогрева, час - constanta

которые были разделены на константы (3.1) и входные параметры (3.2), которые подлежат регулировке

Замечание: все параметры в метрических единицах Си, температура - в К. Когда производится выражение в длугие единицы (для использования соотнешений из некоторых источников) - об этом делается ссылка.

## 3.2 Постоянные параметры (константы), включая параметры сплава ВТ6

In [3]:
DLucha=15e-3                               # диаметр развертки луча, м    
R_1=250e-3                                 # внутренний диаметр сферы - constanta
ro_bt6 = 4430000.                          # плотность сплава ВТ6 г/куб.м
eps = 0.98                                 # к-т использования титана в DED-EBM процессе
q_bt6 = 0.619                              # удельная теплоёмкость сплава ВТ6, Дж/(г*К)
lambda_bt6_0 = 8.8                         # теплопроводность BT6 при 300К, Вт/(м*К)
b = 0.026                                  # к-т теплопроводности, вычислен по справочным данным, Вт/(м*К^2)
T0 = 300.0                                 # температура 20°С (300К)
fi = 5.                                    # шаг деления окружности на сегменты сетки (сеточное представление функции), градусы
TimeShift = 21600.                         # время наплавки всего изделия (6 часов), сек.
qv = 3.125e+4                              # плотность мощности источника теплоты Вт/куб.м (при мощности луча 30 кВт)
delta_tau = 1.0                            # минимальный временной отсчёт, сек
q_delta = 18.8959                          # значение теплового излучения нагретого сплава ВТ6 при t_с_bt6, ВТ/кв.м
l_c_bt6 = 358                              # удельная теплота плавления ВТ6, Дж/г
с = 0.54                                   # удельная тепломкость ВТ6, Дж/гК
d = 0.002                                  # диаметр проволоки, мм      
delta_tau = 1.0                            # шаг дискретезации, сек
DZH = 0.06                                 # диаметр отверстия в изделии, м

## 3.3 Входные параметры (регулируемые) для эксперимента

In [5]:
# для эксперимента исследуется 1-4
elemPerLuch = 1                    # число элементов в развертке луча (дорожек наплавки при формировании стенки), должно быть больше 1 - опеделяет ширину элемента

# для эксперимента исследуется от 1е-3 до 25e-3(?)
V = 8e-3                           # м/сек скорость наплавки

# для эксперимента исследуется от 0.4e-3 до 2e-3(?)
delta = 1.0e-3                     # высота элемента в м

# для эксперимента исследуется от 1923,15K до 2300K(?)
t_с_bt6 = 1923.15                  # температура плавления °К (1650°С -  достигается значением токов), все вычисления - в Кельвинах!

# для эксперимента исследуется от 0,015 до 0,025(?)
h1=15e-3                          # толщина стенки сферы, м

# для эксперимента исследуется от 0.3 до 0.5
Ib = 0.5                           # ток луча, А

# для эксперимента исследуется от 50000 до 60000 В
Ub = 60000                         # ускоряющее напряжение, В

In [6]:
# вычисление некоторыех дополнительных параметров

ElStep=DLucha/elemPerLuch                  # размер геометрического элемента (ширина элемента), м
delta_x = delta_tau * V                    # размер геометрического элемента (длина элемента), м
delta_v = delta_x * ElStep * delta         # объём единичного елемента наплавки, куб.м
delta_m = delta_v * ro_bt6                 # масса единичного элемент анаплавки, г
L = TimeShift * V * elemPerLuch            # длина всего слоя наплавки, м
summ_delta_x = L / delta_x * elemPerLuch   # всего единичных элементов в изделии, шт.                 
M = delta_m * summ_delta_x                 # масса всего изделия, г
T1 = t_с_bt6                               # задаём для расчётом начальную температуру t1 как температуру плавления ВТ6 (но может быть и выше)
a0=0.00025                                 # аппроксимация по точкам (грубая)

In [7]:
h_DZH = math.sqrt((0.5*R_1)**2-(0.5*DZH)**2)   # высота от точки на сечении по диаметру отверстия полусферы на затравке до центра сферы (полусфера наплавляется не с 0 точки)
ky = math.ceil((0.5*R_1 - h_DZH)/delta)        # количество "пропускаемых" слоёв т.к. полусфера наплавляется не с 0 точки (не "от полюса")
n = math.ceil(0.5*R_1/delta) 
r = round(math.sqrt((0.5*R_1)**2 - ((n-ky)*delta)**2), 4)
n,r, h_DZH, ky, DZH

(125, 0.0314, 0.12134661099511597, 4, 0.06)

## 3.4 Вспомогательные функции

In [8]:
# Set seeds to make the experiment more reproducible.
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything(SEED)
warnings.filterwarnings('ignore')

In [ ]:
def plot_temperature_gradient(x, y):

    # выбираем цветовую палитру
    mapper = linear_cmap(field_name="y", palette=Turbo256, low=min(y), high=max(y))

    # create plot
    p = figure(width=1200, height=750)

    # create circle renderer with color mapper
    p.circle(x, y, color=mapper, size=10)

    # обязательная строка, чтобы график рисовался в google colab
    bokeh.io.output_notebook()
    show(p)

In [9]:
# некоторые физические параметры зависят от температуры материала - сразу зададим их

# значение теплопроводности ВТ6 (Вт/(мК))
def lambda_bt6(T1, T0, b):
    return lambda_bt6_0 + (b * (T1 - T0)/2)

# для упрощения иногда применяется среднее значение теплопроводности (в работе не применяется - doesn't use for a while)
lambdabt6 = 18.0

# среднее интегральное значение теплопроводности ВТ6 (косвенно уточняется мощность источника)
def Llambda_bt6(T1i,T2):                         
    if abs(T1i-T2)<0.001:
        z=lambda_bt6(T1i, T0, b)
    else:
        z=(1/(T2-T1i))*(quad(lambda T1i: lambda_bt6(T1i, T0, b), T1i,T2)[0])
    return z

# значение теплоёмкости ВТ6 (Дж/(гК))
def c_bt6(T):
    return a0*T+с


cbt6 = 0.7               # для упрощения возьмём среднее значение теплоёмкости (в работе не применяется - doesn't use for a while)

r1=0.075                 # радиус слоя (здесь для затравки по оси с толщиной стенки 15мм)
l0 = 2*math.pi*r1        # длина начального слоя наплавки (задаётся технологически)
l=l0
T2=T0 
                                        
T3=round(fsolve(lambda T3:T3-T1-(qv*r1**2)/(4*Llambda_bt6(T1,T2)) ,T1)[0],1)
T3                      # заданное значение температуры поверхности затравки

1925.4

In [10]:
# размер выборки - nn (выбирается в ручную)

nn = 2000

# функция генерации случайных значени в рамках диапазона с учётом плотности распределения
def generator(mean, min, max, nn):

  # сгенерируем массив с данными
    if mean-min < (max-min)/2:
        a = 40
    elif mean-min == (max-min)/2:
        a= 0
    else:
        a= -40
        pre = skewnorm.rvs(a = a, # скошенность (skewness)
                           loc = mean, # медиана
                           scale = mean-min, # отклонение от среднего
                           size = n*10, # размер генерируемого массива
                           random_state = 42)
    prev = []
 
  # зададим количество и размер выборок
    n_samples = nn
    sample_size = 10
 
  # в цикле будем формировать выборки
    for i in range(n_samples):
 
  # путем отбора нужного количества элементов из генеральной совокупности
        sample = np.random.choice(pre, sample_size, replace = True)
 
  # для каждой выборки рассчитаем среднее значение и поместим в список выборочных средних
        prev.append(np.mean(np.fabs(sample)))
    return np.array(prev).reshape(-1, 1)